This note book is the raw implementation of the spell check program written by Peter Norvig. This is meant to demonstrate my understanding of the program and the concepts behind every little section of code. To the best of my abilities I will explain the code and why things work.
The link to Peter Norvig's blog is supplied here:
http://norvig.com/spell-correct.html

In [31]:
import re, collections

The program is written with Bayes theorem in mind. That is to say that $c$ represents a correction and $w$ represents a word typed by a user. The program is trying to figure out $argMax_c P(c|w)$, which when expanded is equivalent to saying $argMax_c P(w|c) P(c)$. Or in plain English, the program is trying to find the most probable correction to a word typed out. To do that, the program must first figure out all probable corrections of a given word and then find the most frequent or highest probable correction.

The word function below reads in a text file and reads off all of the words found in the file. It also makes sure to convert everyting into lowercase letters for comparision purposes.

In [32]:
def words(text): return re.findall('[a-z]+', text.lower()) 

The train function takes in a list of words and counts the number of occurences each word has. Thus NWORDS contains the true number of different words as well as the number of occurences each word has.

In [33]:
def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(file('big.txt').read()))

With an alphabet defined the next step is to define quantitative heuristics that will determine "good" canidates for spelling corrections. In the edits1 function, the return value is defined as the set of all canidates that are 1 correctional move away from a proper correction. These moves are deletes, transposes, replaces, and inserts.

In [34]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

The known_edits2 function does the same thing as the edits1 function, but does this process over as smaller subset of possibilites. That is to say, it only brings back the candidates that are both known words and are 2 correctional moves away from a given word. And with all functions up to known_edits2, we have now pretty much calculated all of $P(c)$.

In [35]:
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

The known function returns the set of canidates that are for a fact to be known words as defined by a dictionary. This means that only correct words will be returned. This will be a part of the $P(w|c)$ portion of the program.

In [36]:
def known(words): return set(w for w in words if w in NWORDS)

The correct function then assembles all possible canidates and considers thier editorial distances. It passes through the canidates from least editorial moves to greatest and then returns the most frequently matched canidate as the "correct" word. And thus this function both calculates $P(w|c)$ and returns $argMax_c P(w|c) P(c)$.

In [37]:
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

Below is the program in action.

In [38]:
correct('speling')

'spelling'

In [39]:
correct('spelling')

'spelling'

In [40]:
correct('dod')

'did'

In [41]:
correct('dog')

'dog'